## CSCI 303
# Introduction to Data Science
<p/>
### 13 - Working with SQL Databases (2)

![Relational database icon](sql.png)

## This Lecture
---
- More SELECT queries
  - Operators and functions
  - Sorting
  - Grouping and aggregating
  - Joining tables

The obligatory setup code...

In [1]:
import pandas as pd
import sqlite3   # We'll be using a simple file-based SQLite3 database
from pandas import Series, DataFrame

dburi = 'sqlite:///csci303.sqlite3'

Let's also display info about the tables we'll be using in our examples:

In [2]:
from sqlalchemy import create_engine, inspect
inspector = inspect(create_engine(dburi))

In [3]:
pd.DataFrame(inspector.get_columns('scifi_author'))

""


In [4]:
pd.DataFrame(inspector.get_columns('scifi_work'), columns=['name','type'])

,name,type


## Operators and Functions
---
Wildcard matching using `LIKE`:

In [5]:
pd.read_sql_query("SELECT * FROM scifi_author WHERE name LIKE 'A%%'", dburi) # searches for all authors that start with an A

OperationalError: (sqlite3.OperationalError) no such table: scifi_author
[SQL: SELECT * FROM scifi_author WHERE name LIKE 'A%%']
(Background on this error at: http://sqlalche.me/e/13/e3q8)

The '%' wildcard matches any string of any length, so the above query asks for all authors where the author name starts with an 'A'.

Note that the wildcard in SQL is just a single '%', but that has meaning in Python strings, so we have to escape it by using an extra '%'.

You can also use '_' to stand in for any single character.

So, for instance, we could get authors for whom we have only a first initial:

In [ ]:
pd.read_sql_query("SELECT * FROM scifi_author WHERE name LIKE '_. %%'", dburi)

SQL has numerous functions and operators.  For instance, pretty much any mathematical expression is allowed:

In [ ]:
# gets the name of the author
# also gets the age of the author by subtracting birth year from death yaer (as long as that person is alive)
query = """
 SELECT name, death_year - birth_year AS approx_age 
 FROM scifi_author 
 WHERE death_year IS NOT NULL
"""

pd.read_sql_query(query, dburi)

We slipped in a couple of other SQL things there.  Let's explain those.

First, the `AS` keyword lets us rename or "alias" a column.  That's handy when doing an expression like above.

We'll see other uses for `AS` later.

Also, we used another operator, `IS NOT NULL`.

NULL values in SQL are not comparable - any attempt to compare them using a relational operator (such as =) will always return false!

Instead, use `IS NULL` and `IS NOT NULL` to accept/reject NULL values.

There are a number of useful string functions and operators.

For example, is it "A. E. Van Vogt", or "A. E. van Vogt"?

We can convert strings to all lowercase using `lower`:

In [ ]:
pd.read_sql_query("SELECT * FROM scifi_author WHERE lower(name) LIKE '%%van vogt'", dburi)
#pd.read_sql_query("SELECT * FROM scifi_author WHERE name LIKE '%%van vogt'", dburi)

All of the functions and operators mentioned above are standard SQL.

Your database may supply (many) others in addition.

For example, here's a link to documentation on PostgreSQL's functions and operators:
https://www.postgresql.org/docs/9.5/static/functions.html

## Sorting
---
To sort the results of a query, add an `ORDER BY` clause.

`ORDER BY` is followed by the columns you want to sort by.  Each column name can optionally be followed with `ASC` (the default) or `DESC` to determine whether the sort is ascending or descending.

In [ ]:
pd.read_sql_query("SELECT * FROM scifi_author ORDER BY name", dburi)[:10] # sorts the database by name

In [ ]:
# sorts the age of the author in descending order
query = """
 SELECT name, death_year - birth_year AS approx_age 
 FROM scifi_author 
 WHERE death_year IS NOT NULL
 ORDER BY approx_age DESC 
"""

pd.read_sql_query(query, dburi) 

## DISTINCT
---
Somewhat related to sorting, sometimes you want to get only a unique set of records back.

This is particularly useful when trying to find the unique settings for a particular column.

For instance, our data unfortunately has a lot of duplicates.

Let's see just the unique author names starting with 'C':

In [ ]:
query = """
SELECT DISTINCT name
FROM scifi_author
WHERE name LIKE 'C%%'
"""

pd.read_sql_query(query, dburi)

## Grouping and Aggregating
---
SQL has capabilities for grouping data and computing aggregate functions on the groups, very similar to what we saw pandas doing in the Advanced pandas lecture.

The basic syntax is

```
SELECT a1, a2, ..., fn1(a3), fn2(a4), ...
FROM table
GROUP BY a1, a2, ...;
```

where `fn1` etc. compute some kind of aggregate.  Some of the functions available are `COUNT`, `SUM`, `AVG`, `MIN`, and `MAX`.

It is important that every attribute selected participate in the group operation.

For example, this query is valid and finds out the multiplicity of each author in our database:

In [ ]:
query = """
SELECT name, COUNT(name) FROM scifi_author
GROUP BY name
"""

pd.read_sql_query(query, dburi)

whereas this query is invalid, because we are not grouping by birth_year:

In [ ]:
query = """
SELECT name, COUNT(name), birth_year FROM scifi_author
GROUP BY name
"""
pd.read_sql_query(query, dburi)

Usefully, we can also `ORDER BY` aggregate expressions. Suppose we want the most duplicated authors (to see where we need to do the most cleanup):

In [ ]:
query = """
SELECT name, COUNT(name) FROM scifi_author
GROUP BY name
ORDER BY COUNT(name) DESC
"""
pd.read_sql_query(query, dburi)

We can also use another clause with the keyword `HAVING`, which lets us filter the grouped results according to aggregate values.  Let's only see the authors with multiplicity greater than 2:

In [ ]:
query = """
SELECT name, COUNT(name) FROM scifi_author
GROUP BY name
HAVING COUNT(name) > 2
"""
pd.read_sql_query(query, dburi)

Aggregate functions can also be applied without grouping:

In [ ]:
query = """
 SELECT 
   MIN(death_year - birth_year),
   AVG(death_year - birth_year),
   MAX(death_year - birth_year) 
 FROM scifi_author 
 WHERE death_year IS NOT NULL
"""

pd.read_sql_query(query, dburi)

## Joins
---
Databases are often factored in such a way as to minimize duplicate information.  (This database didn't succeed so well in that.)

For example, we have a table of Science Fiction works, but it doesn't include author names or dates.  

Rather, the authors live in a separate table, and are simply referenced by a key field from the works table.

In [ ]:
pd.DataFrame(inspector.get_columns('scifi_work'), columns=['name','type'])

The `author_id` field provides our linkage to the `scifi_author` table.

There are two ways to do inner joins, which are the joins we most commonly want to do.

Here's the "wordy" way to join `scifi_author` and `scifi_work`:

In [ ]:
query = """
SELECT scifi_author.name, scifi_work.title, scifi_work.publication_year
FROM   scifi_author JOIN scifi_work ON scifi_author.id = scifi_work.author_id
"""

pd.read_sql_query(query, dburi)

Some of the table name specifiers are unnecessary in the above query, as SQL can work out what table you mean if the column name is unique.

The only column we really *need* the specifier on is the `scifi_author.id` column, since there is also a column named `id` in `scifi_work`.

However, it makes things clearer if we do something to specify what tables columns are coming from.

This is a common use of table aliases using the `AS` keyword.

In [ ]:
query = """
SELECT a.name, w.title, w.publication_year
FROM   scifi_author AS a JOIN scifi_work AS w
ON a.id = w.author_id
"""

pd.read_sql_query(query, dburi)

An equivalent, and more compact way to write the same query is to move the join condition(s) into the WHERE clause, and simply list the tables you want data from:



In [ ]:
query = """
SELECT a.name, w.title, w.publication_year
FROM scifi_author AS a, scifi_work AS w
WHERE a.id = w.author_id
"""
pd.read_sql_query(query, dburi)

Now we can ask questions like, "What books did Ann Leckie write?"

In [ ]:
query = """
SELECT a.name, w.title, w.publication_year
FROM scifi_author AS a, scifi_work AS w
WHERE a.id = w.author_id
AND a.name = 'Ann Leckie'
"""
pd.read_sql_query(query, dburi)

Or, "Who wrote *I, Robot*?"

In [ ]:
query = """
SELECT a.name
FROM scifi_author AS a, scifi_work AS w
WHERE a.id = w.author_id
AND w.title = 'I, Robot'
"""
pd.read_sql_query(query, dburi)

## Putting It All Together
---
Now that we know the basics, let's try some queries with the Sci-fi books dataset.

Let's start with, "How many books are listed for each author entry?"

In [ ]:
query = """
SELECT a.id, a.name, COUNT(w.title)
FROM   scifi_author AS a, scifi_work AS w
WHERE  a.id = w.author_id
GROUP BY a.id, a.name
ORDER BY a.name
"""
pd.read_sql_query(query, dburi)

That's probably not what we wanted; our data has some bad duplication in it.

Let's try just grouping by author name:


In [ ]:
query = """
SELECT a.name, COUNT(w.title)
FROM   scifi_author AS a, scifi_work AS w
WHERE  a.id = w.author_id
GROUP BY a.name
ORDER BY a.name
"""
pd.read_sql_query(query, dburi)

I wonder if duplication is a problem in the works data, too?

Let's look closer at one of our more prolific authors:

In [ ]:
query = """
SELECT w.title, COUNT(w.title)
FROM   scifi_author AS a, scifi_work AS w
WHERE  a.id = w.author_id
AND    a.name = 'Arthur C. Clarke'
GROUP BY w.title
HAVING COUNT(w.title) > 1
ORDER BY w.title
"""
pd.read_sql_query(query, dburi)

Hm, maybe we can filter this down a bit.  It turns out you can combine COUNT and DISTINCT - this will at least let us remove books with exact duplicate titles.

(It still won't help with books that are the same but are listed differently, or books in other languages.)

In [ ]:
query = """
SELECT a.name, COUNT(w.title), COUNT(DISTINCT w.title)
FROM   scifi_author AS a, scifi_work AS w
WHERE  a.id = w.author_id
GROUP BY a.name
ORDER BY a.name
"""
pd.read_sql_query(query, dburi)

If we want to see the actual distinct titles, we can just use DISTINCT without grouping:

In [ ]:
query = """
SELECT DISTINCT a.name, w.title
FROM   scifi_author AS a, scifi_work AS w
WHERE  a.id = w.author_id
ORDER BY a.name, w.title
"""
pd.read_sql_query(query, dburi)

"Which of our authors is most prolific?"

In [ ]:
query = """
SELECT a.name, COUNT(DISTINCT w.title) AS unique_count
FROM   scifi_author AS a, scifi_work AS w
WHERE  a.id = w.author_id
GROUP BY a.name
ORDER BY unique_count DESC
"""
pd.read_sql_query(query, dburi)[:5]

"What year had the most works published?"

In [ ]:
query = """
SELECT publication_year, COUNT(DISTINCT title) AS unique_count
FROM scifi_work
GROUP BY publication_year
ORDER BY unique_count DESC
"""
pd.read_sql_query(query, dburi)[:5]

"What years did Arthur C. Clarke publish the most in?"

In [ ]:
query = """
SELECT w.publication_year, COUNT(DISTINCT w.title) AS unique_count
FROM scifi_work AS w, scifi_author AS a
WHERE a.id = w.author_id AND a.name = 'Arthur C. Clarke'
GROUP BY publication_year
ORDER BY unique_count DESC
"""
pd.read_sql_query(query, dburi)[:5]

We clearly don't know more than we know...

What other questions can you think to ask of this data?

## Extra Stuff 
---
- SQL odds and ends
  - Set operations
  - Subqueries
  - Outer joins